***
Zadanie 1.
***

In [11]:
import numpy as np
from dash import Dash, html
from dash import dcc, Output, Input, callback
import plotly.express as px

app = Dash(__name__)

AMPLITUDA_MIN=-1
AMPLITUDA_MAX=1

app.layout = html.Div([
html.Div("Wykres sinusa"),
html.Label("Amplituda"),
dcc.Slider(
    id="amplituda-slider",
    min=0,
    max=1,
    step=0.1,
    value=0.5,
    marks={i /10: str(i / 10) for i in range(11)}
),
html.Label("Czestotliwosc (Hz): "),
dcc.Slider(
    id="czestotliwosc-slider",
    min=1,
    max=100,
    step=1,
    value=10,
    marks={i:str(i) for i in range (0,101,10)}
),
    dcc.Graph(id="sinus-graph")
])

@app.callback(
    Output("sinus-graph","figure"),
    [Input("amplituda-slider","value"),
    Input("czestotliwosc-slider","value")]
)
def update_graph(A, f):

    t = np.linspace(0,1,1000)
    y = A* np.sin(2*np.pi * t * f)

    fig = px.line(x=t, y=y, labels={'x':'Czas(s)','y':'Amplituda'}, title="Wykres sinusa")

    fig.update_yaxes(range=[AMPLITUDA_MIN,AMPLITUDA_MAX])

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

***
Zadanie 2.
***

In [12]:
import pandas as pd
import plotly.express as px
from dash import Dash,html,dcc,Output,Input

df = pd.read_csv("lista_imion.csv")

px.defaults.template = "plotly_white"

app = Dash(__name__)

app.layout = html.Div([
    html.Div("Najczesciej wystepujace imiona"),
    html.Label("Liczba imion do wyswietlenia:"),
    dcc.Slider(
        id="liczba-imion-slider",
        min=3,
        max=20,
        step=1,
        value=10,
        marks={i:str(i) for i in range(3,21)}
    ),
    dcc.Graph(id="najczesciej-wystepujace-imiona")
])

@app.callback(
    Output("najczesciej-wystepujace-imiona","figure"),
    [Input("liczba-imion-slider","value")]
)
def update_bar_chart(n):
    
    top_names = df.nlargest(n, 'LICZBA_WYSTĄPIEŃ')

    fig = px.bar(top_names, x='IMIĘ_PIERWSZE', y='LICZBA_WYSTĄPIEŃ',
                 labels={'IMIĘ_PIERWSZE': 'Imię', 'LICZBA_WYSTĄPIEŃ': 'Liczba wystąpień'},
                 title=f"Najczęściej występujące imiona (Top {n})")

    # Opisanie osi
    fig.update_xaxes(title_text="Imię")
    fig.update_yaxes(title_text="Liczba wystąpień")

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

***
Zadanie 3.
***

In [13]:
import pandas as pd
import plotly.express as px
from dash import Dash, html, dcc, Output, Input
import plotly.graph_objects as go


df = pd.read_csv("prawa_jazdy.csv")


px.defaults.template = "plotly_white"


wojewodztwa = df['WOJEWODZTWO'].unique()

app = Dash(__name__)

app.layout = html.Div([
    html.Div("Histogram nowych praw jazdy"),
    html.Label("Wybierz województwo:"),
    dcc.Dropdown(
        id="wojewodztwo-dropdown",
        options=[{'label': woj, 'value': woj} for woj in wojewodztwa],
        value=wojewodztwa[0]  # Domyślne województwo
    ),
    dcc.Graph(id="histogram-praw-jazdy")
])

@app.callback(
    Output("histogram-praw-jazdy", "figure"),
    [Input("wojewodztwo-dropdown", "value")]
)
def update_histogram(wojewodztwo):
    
    df_selected = df[df['WOJEWODZTWO'] == wojewodztwo]

    
    df_kobiety = df_selected[df_selected['PLEC'] == 'K']
    df_mezczyzni = df_selected[df_selected['PLEC'] == 'M']

    
    fig = go.Figure()

    fig.add_trace(go.Histogram(x=df_kobiety['WIEK'], name='Kobiety', marker_color='rgba(255, 0, 0, 0.6)'))
    fig.add_trace(go.Histogram(x=df_mezczyzni['WIEK'], name='Mężczyźni', marker_color='rgba(0, 0, 255, 0.6)'))

    
    fig.update_layout(barmode='overlay',
                      title=f"Histogram nowych praw jazdy dla {wojewodztwo}",
                      xaxis=dict(title="Wiek"),
                      yaxis=dict(title="Liczba nowych praw jazdy"))

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
